In [ ]:
from struphy.geometry.domains import Cuboid

l1 = -.5
r1 = .5
l2 = -.5
r2 = .5
l3 = 0.
r3 = 1.
domain = Cuboid(l1=l1, r1=r1, l2=l2, r2=r2, l3=l3, r3=r3)

In [ ]:
from struphy.fields_background.generic import GenericCartesianFluidEquilibrium
import numpy as np
T_h = 0.2/6
gamma = 5/3
n_fun = lambda x, y, z: np.sin(2*np.pi*x)*np.sin(4*np.pi*y) + 2
#n_fun = lambda x, y, z: np.exp(-np.cos(2*np.pi*x)+1)*np.exp(-np.cos(2*np.pi*y)+1)
#def n_fun(x,y,z):
 #   ind = x < 0.5
  #  out = np.ones_like(x)*2
   # out[x>0.5] = 0.
    #return out
bckgr = GenericCartesianFluidEquilibrium(n_xyz=n_fun)
bckgr.domain = domain

In [ ]:
x = np.linspace(-.5, .5, 100)
y = np.linspace(-.5, .5, 90)
xx, yy = np.meshgrid(x, y)
eta1 = np.linspace(0, 1, 100)
eta2 = np.linspace(0, 1, 90)
eta3 = np.linspace(0,1,1)
ee1, ee2, ee3 = np.meshgrid(eta1, eta2, eta3, indexing="ij")

In [ ]:
h_values = np.array([((2**k)*10**-3*0.25) for k in range(2, 12)])  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from struphy.pic.particles import ParticlesSPH

nx, ny, nz = 32, 32, 1
Nps = np.array([(2**k)*10**3 for k in range(0, 9)])  
h_values = np.array([((2**k)*10**-3*0.25) for k in range(2, 12)])  

#start = 1e-2  
#end = 0.5
#num_points = 7

# Create the list
#h_values = np.linspace(start, end, num_points)
# Add 1/64 (0.015625) if it's not already in the array
#extra_value = 1/nx
#if not np.isclose(h_values, extra_value).any():
#    h_values = np.append(h_values, extra_value)

# Sort the values to keep them in ascending order
#h_values = np.sort(h_values)
h_arr = np.array(h_values)  # 8 Werte
boxes_per_dim = (nx, ny, nz)
bc = ['periodic'] * 3
eps = 0.5
h3 = 1 / nz

# Analytic solution 
n_analytic = n_fun(xx, yy, 0)
n_sq = np.squeeze(n_analytic).T
error = np.empty((h_arr.size, Nps.size))

# Plot-Setup
n_rows, n_cols = len(h_arr), len(Nps)
fig, axes = plt.subplots(n_rows, n_cols, figsize=(3.5 * n_cols, 3.5 * n_rows), constrained_layout=True)

for i_h, h in enumerate(h_arr):
    for i_n, Np in enumerate(Nps):
        print(f"{h = }, {Np= }")
        h1 = h2 = h

        # generating particles 
        particles = ParticlesSPH(
            bc=bc,
            domain=domain,
            bckgr_params=bckgr,
            Np = Np,
            boxes_per_dim=boxes_per_dim,
            eps=eps,
        )
        particles.draw_markers(sort=False)
        particles.initialize_weights(reject_weights=True, threshold=0.)

        n_sph = particles.eval_density(
            ee1, ee2, ee3,
            h1=h1, h2=h2, h3=h3,
            kernel_type="gaussian_2d",
            fast=True,
        )
        n_sph /= domain.jacobian_det(eta1, eta2, eta3)
        n_sph_sq = np.squeeze(n_sph)

        # Computing the error 
        diff = np.max(np.abs(n_sq - n_sph_sq))
        rel_diff_max = diff / np.max(np.abs(n_sq))
        error[i_h, i_n] = rel_diff_max

        # Plot
        ax = axes[i_h, i_n]
        p = ax.pcolor(ee1[:, :, 0], ee2[:, :, 0], n_sph[:, :, 0], shading='auto')
        ax.set_aspect('equal')
        if i_h == 0:
            ax.set_title(f'Np = {Np}')
        if i_n == 0:
            ax.set_ylabel(f'h = {h:.0e}')
        fig.colorbar(p, ax=ax)

# Analytic solution presented separately 
fig2, ax2 = plt.subplots(figsize=(6, 6))
pc = ax2.pcolormesh(ee1[:, :, 0], ee2[:, :, 0], n_sq, shading='auto')
fig2.colorbar(pc, ax=ax2)
ax2.set_title('Analytic solution')
ax2.set_aspect('equal')

plt.show()


In [ ]:
print(np.max(error))
print(np.min(error))

print(error[:,:])

hh,nn = np.meshgrid(Np, h_arr)

print("error shape:", error.shape)
print("hh shape:", hh.shape)
print("nn shape:", nn.shape)
print(hh)

In [ ]:
np.log2(h_arr)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

hh, nn = np.meshgrid(np.log10(h_arr), np.log10(Nps), indexing='ij')

# 2D Plot
plt.figure(figsize=(6, 6))
plt.pcolor(hh, nn, np.log10(error), shading='auto')
plt.title('Error')
plt.colorbar(label='log10(error)')
plt.xlabel('log10(h)')
plt.ylabel('log10(Np)')


min_indices = np.argmin(error, axis=0)  
min_h_values = np.log10(h_arr[min_indices])  
log_Nps = np.log10(Nps)

# Rote Linie plotten
plt.plot(min_h_values, log_Nps, 'r-', label='Min error h for each Np', linewidth=2)
plt.legend()

plt.show()


import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

hh, nn = np.meshgrid(np.log10(h_arr), np.log10(Nps), indexing='ij')

# 3D Plot
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
surf = ax.plot_surface(hh, nn, np.log10(error), cmap=cm.coolwarm)
fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5)


ax.set_xlabel('log10(h)')
ax.set_ylabel('log10(Np)')
ax.set_zlabel('log10(Error)')
ax.set_title('Error Surface')


min_indices = np.argmin(error, axis=0)  
min_h_values = np.log10(h_arr[min_indices])  
log_Nps = np.log10(Nps)
min_error_values = np.log10(error[min_indices, np.arange(len(Nps))]) 


ax.plot(
    min_h_values,           # X: log10(h)
    log_Nps,                # Y: log10(Np)
    min_error_values,       # Z: log10(error)
    color='red',
    linewidth=4,
    label='Min Error Line',
    zorder=10,              # Draw on top
    alpha=1.              # Fully opaque
)

ax.legend()
plt.show()


In [ ]:
h_values = np.array([((2**k)*10**-3*0.25) for k in range(7, 12)])  
np.log10(h_values)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from struphy.pic.particles import ParticlesSPH

nx, ny, nz = 32, 32, 1
Nps = np.array([(2**k)*10**3 for k in range(0, 9)])  
#h_values = np.array([((2**k)*10**-3*0.25) for k in range(7, 12)])   

start = 2**7*10**-3*0.25
end = 2**12*10**-3*0.25
num_points = 70
h_values = np.linspace(start, end, num_points)
# Create the list
#h_values = np.linspace(start, end, num_points)
# Add 1/64 (0.015625) if it's not already in the array
#extra_value = 1/nx
#if not np.isclose(h_values, extra_value).any():
#    h_values = np.append(h_values, extra_value)

# Sort the values to keep them in ascending order
#h_values = np.sort(h_values)
h_arr = np.array(h_values)  # 8 Werte
boxes_per_dim = (nx, ny, nz)
bc = ['periodic'] * 3
eps = 0.5
h3 = 1 / nz

# Analytic solution 
n_analytic = n_fun(xx, yy, 0)
n_sq = np.squeeze(n_analytic).T
error1 = np.empty((h_arr.size))
error2 = np.empty((h_arr.size))
error3 = np.empty((h_arr.size))



for i_h, h in enumerate(h_arr):
    Np1 = 10*10**3
    #h1 = h2 = h

    # generating particles 
    particles = ParticlesSPH(
        bc=bc,
        domain=domain,
        bckgr_params=bckgr,
        Np = Np1,
        boxes_per_dim=boxes_per_dim,
        eps=eps,
    )
    particles.draw_markers(sort=False)
    particles.initialize_weights(reject_weights=True, threshold=0.)

    n_sph = particles.eval_density(
        ee1, ee2, ee3,
        h1=h, h2=h, h3=h3,
        kernel_type="gaussian_2d",
        fast=True,
    )
    n_sph /= domain.jacobian_det(eta1, eta2, eta3)
    n_sph_sq = np.squeeze(n_sph)

    # Computing the error 
    diff = np.max(np.abs(n_sq - n_sph_sq))
    rel_diff_max = diff / np.max(np.abs(n_sq))
    error1[i_h] = rel_diff_max


for i_h, h in enumerate(h_arr):
    Np2 = 100*10**3
    #h1 = h2 = h

    # generating particles 
    particles = ParticlesSPH(
        bc=bc,
        domain=domain,
        bckgr_params=bckgr,
        Np = Np2,
        boxes_per_dim=boxes_per_dim,
        eps=eps,
    )
    particles.draw_markers(sort=False)
    particles.initialize_weights(reject_weights=True, threshold=0.)

    n_sph = particles.eval_density(
        ee1, ee2, ee3,
        h1=h, h2=h, h3=h3,
        kernel_type="gaussian_2d",
        fast=True,
    )
    n_sph /= domain.jacobian_det(eta1, eta2, eta3)
    n_sph_sq = np.squeeze(n_sph)

    # Computing the error 
    diff = np.max(np.abs(n_sq - n_sph_sq))
    rel_diff_max = diff / np.max(np.abs(n_sq))
    error2[i_h] = rel_diff_max
    
    

for i_h, h in enumerate(h_arr):
    Np3 = 500*10**3
    #h1 = h2 = h

    # generating particles 
    particles = ParticlesSPH(
        bc=bc,
        domain=domain,
        bckgr_params=bckgr,
        Np = Np3,
        boxes_per_dim=boxes_per_dim,
        eps=eps,
    )
    particles.draw_markers(sort=False)
    particles.initialize_weights(reject_weights=True, threshold=0.)

    n_sph = particles.eval_density(
        ee1, ee2, ee3,
        h1=h, h2=h, h3=h3,
        kernel_type="gaussian_2d",
        fast=True,
    )
    n_sph /= domain.jacobian_det(eta1, eta2, eta3)
    n_sph_sq = np.squeeze(n_sph)

    # Computing the error 
    diff = np.max(np.abs(n_sq - n_sph_sq))
    rel_diff_max = diff / np.max(np.abs(n_sq))
    error3[i_h] = rel_diff_max


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))

plt.loglog(h_arr, error1, label="Np = 50,000", marker='o')
plt.loglog(h_arr, error2, label="Np = 100,000", marker='s')
plt.loglog(h_arr, error3, label="Np = 400,000", marker='^')

plt.xlabel("Smoothing length h", fontsize=12)
plt.ylabel("Max relative error", fontsize=12)
plt.title("SPH density estimation error vs h", fontsize=14)
plt.legend()
plt.grid(True, which="both", ls="--")
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from struphy.pic.particles import ParticlesSPH


start = 2**7*10**-3*0.25
end = 2**12*10**-3*0.25
num_points = 70
h_values = np.linspace(start, end, num_points)
h_arr = np.array(h_values)  # 8 Werte
#boxes_per_dim = (8,8, 1)
boxes_per_dim = (nx, ny, nz)
bc = ['periodic'] * 3
eps = 0.5
h3 = 1 / nz

# Analytic solution 
n_analytic = n_fun(xx, yy, 0)
n_sq = np.squeeze(n_analytic).T
error1 = np.empty((h_arr.size))
error2 = np.empty((h_arr.size))
error3 = np.empty((h_arr.size))
error4 = np.empty((h_arr.size))



for i_h, h in enumerate(h_arr):
    Np1 = 500*10**3
    #h1 = h2 = h
    nx, ny, nz = 1, 1, 1
    # generating particles 
    particles = ParticlesSPH(
        bc=bc,
        domain=domain,
        bckgr_params=bckgr,
        Np = Np1,
        boxes_per_dim=boxes_per_dim,
        eps=eps,
    )
    particles.draw_markers(sort=False)
    particles.initialize_weights(reject_weights=True, threshold=0.)

    n_sph = particles.eval_density(
        ee1, ee2, ee3,
        h1=h, h2=h, h3=h3,
        kernel_type="gaussian_2d",
        fast=True,
    )
    n_sph /= domain.jacobian_det(eta1, eta2, eta3)
    n_sph_sq = np.squeeze(n_sph)

    # Computing the error 
    diff = np.max(np.abs(n_sq - n_sph_sq))
    rel_diff_max = diff / np.max(np.abs(n_sq))
    error1[i_h] = rel_diff_max


for i_h, h in enumerate(h_arr):
    Np2 = 500*10**3
    #h1 = h2 = h
    nx, ny, nz = 32, 32, 1
    
    # generating particles 
    particles = ParticlesSPH(
        bc=bc,
        domain=domain,
        bckgr_params=bckgr,
        Np = Np2,
        boxes_per_dim=boxes_per_dim,
        eps=eps,
    )
    particles.draw_markers(sort=False)
    particles.initialize_weights(reject_weights=True, threshold=0.)

    n_sph = particles.eval_density(
        ee1, ee2, ee3,
        h1=h, h2=h, h3=h3,
        kernel_type="gaussian_2d",
        fast=True,
    )
    n_sph /= domain.jacobian_det(eta1, eta2, eta3)
    n_sph_sq = np.squeeze(n_sph)

    # Computing the error 
    diff = np.max(np.abs(n_sq - n_sph_sq))
    rel_diff_max = diff / np.max(np.abs(n_sq))
    error2[i_h] = rel_diff_max
    
for i_h, h in enumerate(h_arr):
    Np3 = 500*10**3
    #h1 = h2 = h
    nx, ny, nz = 64, 64, 1
    
    # generating particles 
    particles = ParticlesSPH(
        bc=bc,
        domain=domain,
        bckgr_params=bckgr,
        Np = Np3,
        boxes_per_dim=boxes_per_dim,
        eps=eps,
    )
    particles.draw_markers(sort=False)
    particles.initialize_weights(reject_weights=True, threshold=0.)

    n_sph = particles.eval_density(
        ee1, ee2, ee3,
        h1=h, h2=h, h3=h3,
        kernel_type="gaussian_2d",
        fast=True,
    )
    n_sph /= domain.jacobian_det(eta1, eta2, eta3)
    n_sph_sq = np.squeeze(n_sph)

    # Computing the error 
    diff = np.max(np.abs(n_sq - n_sph_sq))
    rel_diff_max = diff / np.max(np.abs(n_sq))
    error3[i_h] = rel_diff_max
    
    
for i_h, h in enumerate(h_arr):
    Np4 = 500*10**3
    #h1 = h2 = h
    nx, ny, nz = 128, 128, 1
    
    # generating particles 
    particles = ParticlesSPH(
        bc=bc,
        domain=domain,
        bckgr_params=bckgr,
        Np = Np4,
        boxes_per_dim=boxes_per_dim,
        eps=eps,
    )
    particles.draw_markers(sort=False)
    particles.initialize_weights(reject_weights=True, threshold=0.)

    n_sph = particles.eval_density(
        ee1, ee2, ee3,
        h1=h, h2=h, h3=h3,
        kernel_type="gaussian_2d",
        fast=True,
    )
    n_sph /= domain.jacobian_det(eta1, eta2, eta3)
    n_sph_sq = np.squeeze(n_sph)

    # Computing the error 
    diff = np.max(np.abs(n_sq - n_sph_sq))
    rel_diff_max = diff / np.max(np.abs(n_sq))
    error4[i_h] = rel_diff_max
    

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))

plt.loglog(h_arr, error1, label='boxes_per_dim = (1, 1, 1)', marker='o')
plt.loglog(h_arr, error2, label='boxes_per_dim = (32, 32, 1)', marker='s')
plt.loglog(h_arr, error3, label='boxes_per_dim = (64, 64, 1)', marker='x')
plt.loglog(h_arr, error4, label='boxes_per_dim = (128, 128, 1)', marker='v')

plt.xlabel('Smoothing length $h$')
plt.ylabel('Relative Max Error')
plt.title('Error vs. Smoothing Length for Different Grid Resolutions')
plt.legend()
plt.grid(True, which='both', ls='--')
plt.tight_layout()
plt.show()

    
   



In [ ]:
plt.figure(figsize=(10, 6))

plt.plot(h_arr, error1, label='boxes_per_dim = (1, 1, 1)', marker='o')
plt.plot(h_arr, error2, label='boxes_per_dim = (32, 32, 1)', marker='s')
plt.plot(h_arr, error3, label='boxes_per_dim = (64, 64, 1)', marker='x')
plt.plot(h_arr, error4, label='boxes_per_dim = (128, 128, 1)', marker='v')

plt.xlabel('Smoothing length $h$')
plt.ylabel('Relative Max Error')
plt.title('Error vs. Smoothing Length for Different Grid Resolutions')
plt.legend()
plt.grid(True, which='both', ls='--')
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
#boxen geben auflösung, die nicht unterschreit bar ist 
from struphy.pic.particles import ParticlesSPH
import matplotlib.pyplot as plt 
nx = 32
ny = 32 
nz = 1
Np = [2**k for k in range(3, 11)]
Np = np.array(Np)
h1 = 1/nx
h2 = 1/ny 
h3 = 1/nz
boxes_per_dim = (nx, ny, nz)
bc = ['periodic']*3
eps = 0.5

error = np.empty((Np.size))  # Fehler-Matrix (hx-Zeilen, Np-Spalten)
n_analytic = n_fun(xx, yy, 0)
shape = np.shape(n_analytic)
print(shape)
n_sq= np.squeeze(n_analytic)
n_sq = n_sq.T
plt.figure(figsize = (12,16))
for i_n, ppb in enumerate(Np):      # Index i_n für Np
    

    particles_Np = ParticlesSPH(
        bc=bc,
        domain=domain,
        bckgr_params=bckgr,
        ppb=ppb,
        boxes_per_dim=boxes_per_dim,
        eps=eps,
    )
    particles_Np.draw_markers(sort=False)

    threshold = 0.
    particles_Np.initialize_weights(reject_weights=True, threshold=threshold)

    n_sph_Np = particles_Np.eval_density(
        ee1, ee2, ee3,
        h1=h1, h2=h2, h3=h3,
        kernel_type="gaussian_2d",
        fast=True,
    )
    
    #A = domain.jacobian_det(eta1,eta2,eta3)
    #print(A)
    n_sph_Np /= domain.jacobian_det(eta1,eta2,eta3)
    n_sph_sq = np.squeeze(n_sph_Np)
    
    plt.subplot(5,2, i_n + 1)
    plt.pcolor(ee1[:,:,0], ee2[:,:,0], n_sph_Np[:,:,0])
    plt.axis('square')
    plt.title(f'{ppb=}')
    plt.colorbar()
    
    

    diff = np.max(np.abs(n_sq  - n_sph_sq))
    rel_diff_max = diff/np.max(np.abs(n_sq))
    

    error[i_n] = rel_diff_max  # Speichern des Fehlers
plt.subplot(5,2, 10)
plt.pcolor(ee1[:,:,0], ee2[:,:,0], n_sq)
plt.axis('square')
plt.title('n_xyz')
plt.colorbar()
    
    
import matplotlib.pyplot as plt

# Assuming you have already calculated the `Np` and `error` arrays
plt.figure(figsize=(10, 6))
plt.plot(Np, error, label="Error", marker='o', color='b')

plt.xlabel('Number of Particles per Box (Np)', fontsize=14)
plt.ylabel('L2 Norm of Error', fontsize=14)
plt.title('Error vs Np', fontsize=16)
plt.grid(True)
plt.legend()

plt.show()
